앙상블(ensemble) 기법이란 여러 개의 분류 모델을 조합해서 더 나은 성능을 내는 방법이다.
최근접 이웃, 서포트 벡터 머신, 의사결정 트리, 나이브 베이즈 분류 모델을 실습했는데 지금까지 실습한 여러 분류 모델을 조합해서 단일 모델보다 더 좋은 성능을 낸다.
***

배깅(bagging)의 어원은 부트스트랩(bootstrap)과 어그리게이팅(aggregating, 집계)에서 왔고 상당히 연주하기 힘든 바이올린 연주곡을 두세 명의 초급 바이올린 연주자가 나누어 연주함으로써 한 명의 중급 바이올린 연주자가 연주하는 것 보다 더 나은 연주를 할 수 있는 것과 유사하다.  

중급 바이올린 연주자는 나름 잘 학습했지만 학습 데이터에 과대적합된 의사결정 트리라 볼 수 있다. 의사결정 트리의 단점은 쉽게 과대적합된다는 것인데 배깅은 과대적합이 쉬운 모델에 상당히 적합한 앙상블이다.  

배깅은 한 가지 분류 모델을 여러 개 만들어서 서로 다른 학습 데이터로 학습시킨 후(부트스트랩), 동일한 테스트 데이터에 대한 서로 다른 예측값들을 투표를 통해(어그리게이팅) 가장 높은 예측값으로 최종 결론을 내리는 앙상블 기법이다.
***
랜덤 포레스트(Random Forest)
여러 의사결정 트리를 배깅을 적용해서 예측을 실행하는 모델이다.  

배깅이 모든 분류 모델에서 적용 가능하지만 특히 과대적합되기 쉬운 의사결정 트리에 적용하면 확실히 과대적합을 줄여 성능이 높아지는 혜택을 보기 때문에 배깅은 많은 의사결정 트리 모델의 개선을 이뤘고 여러 개의 나무들이 모여있다는 개념에서 랜덤 포레스트라는 이름이 생겨났다.  

의사결정 트리에서 최적의 특징으로 트리를 분기하는 반면 랜덤 포레스트는 각 노드에 주어진 데이터를 샘플링해서 일부 데이터를 제외한 채 최적의 특징을 찾아 트리를 분기한다. 이러한 과정에서 랜덤 포레스트는 또 한 번 모델의 편향을 증가시켜 과대적합의 위험을 감소시킨다.  
 ***
부스팅(Boosting)
여러 개의 분류기를 만들어 투표를 통해 예측값을 결정한다는 측면에서는 배깅과 동일하다.
배깅은 서로 다른 알고리즘에 기반한 여러 분류기를 병렬적으로 학습하는 반면에 부스팅은 동일한 알고리즘의 분류기를 순차적으로 학습해서 여러 개의 분류기를 만든 후 테스트할 때 가중 투표를 통해 예측값을 결정한다.

In [2]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [14]:
# 사이킷런이 제공하는 MNiST 손글씨 데이터를 불러오기 위해 import 한다.
from sklearn.datasets import load_digits
# 교차 검증을 실행하기 위해서 import 한다.
from sklearn.model_selection import cross_val_score
# 사이킷런의 의사 결정 트리를 사용하기 위해 import 한다.
from sklearn.tree import DecisionTreeClassifier
# 사이킷 런의 랜덤 포레스트를 사용하기 위해 import 한다
from sklearn.ensemble import RandomForestClassifier

MNIST 손글씨 데이터셋으로 랜덤 포레스트 모델과 의사결정 트리 모델을 동일하게 학습시켜 두 모델의 성능 차이를 시각화해 비교한다.  

MNIST 손글씨 데이터 불러오기

In [10]:
mnist = load_digits() # 사이킷 런이 제공하는 MNIST 손글씨 데이터를 불러온다.
# print(mnist)
# features = mnist.data
# labels = mnist.target
features, labels = mnist.data, mnist.target
print(features.shape, labels.shape)

(1797, 64) (1797,)


In [13]:
print(type(features[0]))
print(features[0].reshape(8,8),labels[0])

<class 'numpy.ndarray'>
[[ 0.  0.  5. 13.  9.  1.  0.  0.]
 [ 0.  0. 13. 15. 10. 15.  5.  0.]
 [ 0.  3. 15.  2.  0. 11.  8.  0.]
 [ 0.  4. 12.  0.  0.  8.  8.  0.]
 [ 0.  5.  8.  0.  0.  9.  8.  0.]
 [ 0.  4. 11.  0.  1. 12.  7.  0.]
 [ 0.  2. 14.  5. 10. 12.  0.  0.]
 [ 0.  0.  6. 13. 10.  0.  0.  0.]] 0


교차 검증  

cross_val_score(model, X, Y, scoring=None, cv=None)  
model: 분석 모델  
X: 독립 변수, 학습 데이터(피쳐)  
Y: 종속 변수, 학습 결과(실제값, 레이블, 타겟)  
scoring: 성능 검증에 사용할 평가 지표를 지정한다. => 정확도를 의미하는 'accuracy'를 지정한다.  
cv: 교차 검증 생성기 객체 또는 숫자, None이면 KFold(3), 숫자를 지정하면 KFold(숫자)

MNIST 손글씨 데이터에 대한 의사 결정 트리와 랜덤 포레스트의 검증 정확도를 계산하는 함수


In [34]:
# cross_validation(분류기 모델, 학습 데이터 , 학습데이터에 따른 레이블)
def cross_validation(classifier, features, labels):
    cv_scores = []
    for k in range(10):
        scores = cross_val_score(classifier, features, labels, scoring='accuracy', cv=10)
       # print(scores, scores.mean())
        cv_scores.append(scores.mean())
    return cv_scores

의사 결정 트리의 MNIST 손글씨 검증 정확도를 계산한다.

In [35]:
dt_cv_scores = cross_validation(DecisionTreeClassifier(), features , labels)

랜덤 포레스트의 MNIST 손글씨 검증 정확도를 계싼한다.

In [37]:
rf_cv_scores = cross_validation(RandomForestClassifier(), features , labels)

In [ ]:
for i in range(len(rf_cv_scores)):
    print('{:2d}. {:7.5f}'.format())